In [321]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)

long_positions = []
short_positions = []
cash = 0

# Create an empty DataFrame with 5 columns
df = pd.DataFrame(columns=['Long/Short', 'Qty', 'Price', 'Activity', 'Long Positions', 'Short Positions', 'Cash', 'Avg. Long Price', 'Total Longs', 'Total Shorts', 'Exit Price', 'Breakeven'])

# Create a dictionary with sample data
report_dict = {
    'Long/Short': [],
    'Qty': [],
    'Price': [],
    'Activity': [],
    'Long Positions': [],
    'Short Positions': [],
    'Cash':[],
    'Avg. Long Price': [],
    'Avg. Short Price': [],
    'Total Longs':[], 
    'Total Shorts':[],
    'Exit Price':[],
    'Breakeven':[],
}

start = 2100
end = 2010
step = 10
target = 3

last_entry = None

if start > end:
    prices = np.arange(end, start+step, step)
    prices = np.flip(prices)
else:
    prices = np.arange(start, end+step, step)

def report(LS, Q,P,A,LP,SP, cash, exitprice):
    avg_long = calculateAvgPrice(LP)
    avg_short=calculateAvgPrice(SP)
    qty_long = calculateTotalQty(LP)
    qty_short=calculateTotalQty(SP)
    breakeven=round(((avg_long*qty_long)-(avg_short*qty_short))/(qty_long-qty_short),2)
    report_dict['Long/Short'].append(LS)
    report_dict['Qty'].append(Q)
    report_dict['Price'].append(P)
    report_dict['Activity'].append(A)
    report_dict['Long Positions'].append(stylizePositionList(LP))
    report_dict['Short Positions'].append(stylizePositionList(SP))
    report_dict['Cash'].append(cash)
    report_dict['Avg. Long Price'].append(avg_long)
    report_dict['Avg. Short Price'].append(avg_short)
    report_dict['Total Longs'].append(calculateTotalQty(LP))
    report_dict['Total Shorts'].append(calculateTotalQty(SP))
    report_dict['Exit Price'].append(breakeven+target)
    report_dict['Breakeven'].append(breakeven)
    
def stylizePositionList(positions):
    stylised_str=""
    for pos in positions:
        stylised_str+=f" ({pos[1]}@{pos[0]}) "
    return stylised_str

def calculateAvgPrice(positions):
    qty = 0
    price=0
    for pos in positions:
        qty+=pos[1]
        price+=pos[0]*pos[1]
    return round(price/qty, 2) if qty>0 else 0

def calculateTotalQty(positions):
    qty = 0
    for pos in positions:
        qty+=pos[1]
    return qty




In [322]:
exitprice=0

for price in prices:
    if(last_entry and price-last_entry<=-10):
        # close short positions if in profit
        for short in short_positions:
            if short[0] > price:
                cash+= (short[0]-price)*short[1]
                # logging trade
                short_positions.pop(0)
                report("Closing Short", short[1], price, "Booking Profit From Shorts.", long_positions, short_positions, cash, exitprice)

        # close long if having sufficient cash        
        while cash>0:
            long_0_price, long_0_qty = long_positions[0]
            loss_initial_position = long_0_price-price
            if cash>loss_initial_position:
                qty_to_close = cash//loss_initial_position
                if long_0_qty<=qty_to_close:
                    cash-=long_0_qty*loss_initial_position
                    long_positions.pop(0)
                    report("Closing Long", long_0_qty, price, "Booking Longs bought @ {0}. Closed {1} qty @ {2}".format(long_0_price,long_0_qty, price), long_positions, short_positions, cash, exitprice)
                else:
                    cash-=qty_to_close*loss_initial_position
                    long_positions[0] = (long_0_price, long_0_qty-qty_to_close)
                    report("Closing Long", qty_to_close, price, "Booking Longs bought @ {0}. Closed {1} qty @ {2}".format(long_0_price,qty_to_close, price), long_positions, short_positions, cash, exitprice)
            else:
                break
        # enter long
        long_positions.append((price, 5))
        report("Entering Long", 5, price, "Bought Longs {0} qty  @ {1}.".format(5, price), long_positions, short_positions, cash, exitprice)
        total_longs = sum(j for i, j in long_positions)
        last_entry=price
        # enter short, if 10 points down from last entry
        if(len(long_positions)>=2 and long_positions[-2][0]-price>=10):
            short_qty = total_longs-5
            short_positions.append((price, short_qty))
            report("Entering Short", short_qty, price, "Shorted {0} qty  @ {1}.".format(short_qty, price), long_positions, short_positions, cash, exitprice)
    if not last_entry:
        # enter long
        long_positions.append((price, 5))
        report("Entering Long", 5, price, "Bought Longs {0} qty  @ {1}.".format(5, price), long_positions, short_positions, cash, exitprice)
        total_longs = sum(j for i, j in long_positions)
        last_entry = price

In [323]:
# Add data from the dictionary to the DataFrame
df = pd.concat([df,pd.DataFrame(report_dict)], ignore_index=True)
df['id'] = range(1, len(df) + 1)
df = df[['id', 'Long/Short', 'Qty', 'Price', 'Activity', 'Long Positions', 'Short Positions', 'Cash', 'Avg. Long Price', 'Avg. Short Price', 'Total Longs', 'Total Shorts', 'Exit Price', 'Breakeven']]

In [324]:
df.head(500)

,id,Long/Short,Qty,Price,Activity,Long Positions,Short Positions,Cash,Avg. Long Price,Avg. Short Price,Total Longs,Total Shorts,Exit Price,Breakeven
0,1,Entering Long,5,2100,Bought Longs 5 qty @ 2100.,(5@2100),,0,2100.00,0.0,5,0,2103.00,2100.00
1,2,Entering Long,5,2090,Bought Longs 5 qty @ 2090.,(5@2100) (5@2090),,0,2095.00,0.0,10,0,2098.00,2095.00
2,3,Entering Short,5,2090,Shorted 5 qty @ 2090.,(5@2100) (5@2090),(5@2090),0,2095.00,2090.0,10,5,2103.00,2100.00
3,4,Closing Short,5,2080,Booking Profit From Shorts.,(5@2100) (5@2090),,50,2095.00,0.0,10,0,2098.00,2095.00
4,5,Closing Long,2,2080,Booking Longs bought @ 2100. Closed 2 qty @ 2080,(3@2100) (5@2090),,10,2093.75,0.0,8,0,2096.75,2093.75
5,6,Entering Long,5,2080,Bought Longs 5 qty @ 2080.,(3@2100) (5@2090) (5@2080),,10,2088.46,0.0,13,0,2091.46,2088.46
6,7,Entering Short,8,2080,Shorted 8 qty @ 2080.,(3@2100) (5@2090) (5@2080),(8@2080),10,2088.46,2080.0,13,8,2105.00,2102.00
7,8,Closing Short,8,2070,Booking Profit From Shorts.,(3@2100) (5@2090) (5@2080),,90,2088.46,0.0,13,0,2091.46,2088.46
8,9,Closing Long,3,2070,Booking Longs bought @ 2100. Closed 3 qty @ 2070,(5@2090) (5@2080),,0,2085.00,0.0,10,0,2088.00,2085.00
9,10,Entering Long,5,2070,Bought Longs 5 qty @ 2070.,(5@2090) (5@2080) (5@2070),,0,2080.00,0.0,15,0,2083.00,2080.00


In [325]:
df.to_html('report.html', index=False)
